<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [99]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import sys
import os

In [98]:
# brings in picture data
forests = []
for pic in os.listdir('./data/forest/'):
    forests.append(np.array(image.load_img(path=f'./data/forest/{pic}',
                                          color_mode='rgb', 
                                          target_size=(224,224)
                                          )))
    
mountains = []
for pic in os.listdir('./data/mountain/'):
    mountains.append(np.array(image.load_img(path=f'./data/mountain/{pic}',
                                            color_mode='rgb', 
                                            target_size=(224,224)
                                            )))
    
forests = np.array(forests)

mountains = np.array(mountains)

# creates x variable from pictures
x0 = np.append(forests, mountains, axis=0)

In [67]:
# creates y variables for pictures
y_forests = np.zeros((forests.shape[0], 1))

y_mountains = np.ones((mountains.shape[0], 1))

# creatse y variable
y0 = np.append(y_forests, y_mountains, axis=0)

In [68]:
# converts values to float for model

x0 = x0.astype(float)
y0 = y0.astype(float)

In [69]:
# shuffels data so mountain and forest is mixed
x, y = shuffle(x0, y0)

# perfroms train test split on data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [106]:
# import model and defines input shape
resnet = ResNet50(input_shape=(224, 224, 3), 
                  weights='imagenet', 
                  include_top=False)

# makes layers untrainable
for layer in resnet.layers:
    layer.trainable = False
    
# creates model
resnet_output = resnet.output
flat = Flatten()(resnet_output) # Flattens output to 1 value
predictions = Dense(1, activation='sigmoid')(flat)
mod = Model(resnet.input, predictions)
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [107]:
# fits model to train last trainable layer
mod.fit(x_train, y_train)

Train on 631 samples
631/631 [==============================] - 33s 52ms/sample - loss: 0.6869 - accuracy: 0.9002


In [110]:
score = mod.evaluate(x_test, y_test, verbose=0)
print(f'accuracy = {score[1]}')

accuracy = 0.9154929518699646


# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language